## Baichuan2-7B-chat 如何对接Langchain中 langchain.llms.base 的 LLM 模块，完成本地调用接口，并且提供一个快捷搭建向量数据库、Agent等多功能的Langchain应用的部署方案
### 显卡信息
24G RTX 3090
### 升级pip
python -m pip install --upgrade pip
### 更换清华的源
pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
### 安装依赖
pip install modelscope==1.9.5
pip install transformers==4.35.2
pip install streamlit==1.24.0
pip install sentencepiece==0.1.99
pip install accelerate==0.24.1
pip install langchain==0.0.292

In [ ]:
# 将模型下载至本地/hy-tmp/llm目录下，注意模型大小为15 GB，下载模型大概需要10~20分钟
import torch
from modelscope import snapshot_download, AutoModel, AutoTokenizer
import os
model_dir = snapshot_download('baichuan-inc/Baichuan2-7B-Chat',cache_dir='/hy-tmp/llm', revision='v1.0.4')

In [16]:
# 加载所需的依赖包
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch

In [17]:
# 用本地部署的Baichuan2-7B-chat，自定义一个LLM类，将Baichuan2-7B-chat接入到LangChain框架中。
# 完成自定义LLM类之后，可以以完全一致的方式调用LangChain的接口，而无需考虑底层模型调用的不一致。
# 基于本地部署的Baichuan2-7B-chat自定义 LLM 类并不复杂，我们只需从LangChain.llms.base.LLM 类继承一个子类，并重写构造函数与 _call 函数即可：
class baichuan2_LLM(LLM):
    # 本地Baichuan自定义LLM 类
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, model_path :str):
        # model_path: Baichuan-7B-chat模型路径
        # 从本地初始化模型
        super().__init__()
        print("从本地加载模型...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True,torch_dtype=torch.bfloat16,  device_map="auto")
        self.model.generation_config = GenerationConfig.from_pretrained(model_path)
        self.model = self.model.eval()
        print("完成本地模型加载")

    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
         # 重写调用函数
        messages = [
            {"role": "user", "content": prompt}
        ]
         # 重写调用函数
        response= self.model.chat(self.tokenizer, messages)
        return response
        
    @property
    def _llm_type(self) -> str:
        return "baichuan2_LLM"

In [18]:
llm = baichuan2_LLM('/hy-tmp/llm/baichuan-inc/Baichuan2-7B-Chat')
llm('中国的首都是哪里')

从本地加载模型...


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.45s/it]


完成本地模型加载


'中国的首都是北京。'

In [12]:
llm = baichuan2_LLM('/hy-tmp/llm/baichuan-inc/Baichuan2-7B-Chat')
llm('上海在哪里')

正在从本地加载模型...


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.05s/it]


完成本地模型的加载


'上海市是中国的一个直辖市，位于长江入海口附近，东临东海，西靠江苏和浙江两省，北界江苏和浙江，南濒杭州湾。上海的地理坐标为北纬31°14′，东经121°29′。'